## 3. Data Preparation

This script merges all the currency files and also merge the daily level crypto prices and market prices (dow jones indicator).

Furthermore, custom variables are created such as polarity, Bullish Ratio and Negative Share.

**Output is the daily level summarised data required for modeling exercise**

In [1]:
import pandas as pd
import numpy as np
import string
# !pip install --user -U nltk # if you don't have it
import nltk 
#nltk.download() # if you don't have it
from nltk.corpus import stopwords
import nltk.tokenize 
from nltk.stem import WordNetLemmatizer

#word count
from collections import Counter

#word clouds
import heapq
#from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import collections
import matplotlib.cm as cm
from matplotlib import rcParams
from tqdm import tqdm

#sentiment analysis
from textblob import TextBlob

#text preprocessing
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('wordnet')
from scipy import stats

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saggarwal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Cryptocurrency Prices

Daily level crypto prices are downloaded from internet across different exchanges and average prices are calculated to get final bitcoin prices 

In [2]:
Bitfinex_BTCUSD_1h = pd.read_csv('Bitfinex_BTCUSD_1h.csv', skiprows=1)
Bitfinex_BTCUSD_1h.head()

,unix,date,symbol,open,high,low,close,Volume USD,Volume BTC
0,1629504000000,2021-08-21 00:00:00,BTC/USD,49333.0,49374.0,49030.000000,49043.000000,7.577243e+06,154.502024
1,1629500400000,2021-08-20 23:00:00,BTC/USD,49223.0,49373.0,49049.996934,49330.497131,9.385369e+06,190.254907
2,1629496800000,2021-08-20 22:00:00,BTC/USD,49336.0,49375.0,48977.000000,49223.000000,1.347543e+07,273.762974
3,1629493200000,2021-08-20 21:00:00,BTC/USD,48660.0,49400.0,48660.000000,49337.000000,1.575885e+07,319.412366
4,1629489600000,2021-08-20 20:00:00,BTC/USD,48707.0,48844.0,48644.597808,48664.800914,5.058329e+06,103.942260


In [3]:
Bitstamp_BTCUSD_1h = pd.read_csv('Bitstamp_BTCUSD_1h.csv', skiprows=1)
gemini_BTCUSD_1hr = pd.read_csv('gemini_BTCUSD_1hr.csv', skiprows=1)
gemini_BTCUSD_1hr.head()


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1629504000000,2021-08-21 00:00:00,BTCUSD,49328.89,49383.31,49287.77,49312.94,10.695314
1,1629500400000,2021-08-20 23:00:00,BTCUSD,49242.38,49372.04,49064.75,49328.89,43.297139
2,1629496800000,2021-08-20 22:00:00,BTCUSD,49349.97,49397.11,48985.46,49242.38,89.115682
3,1629493200000,2021-08-20 21:00:00,BTCUSD,48672.49,49400.00,48663.12,49349.97,78.196379
4,1629489600000,2021-08-20 20:00:00,BTCUSD,48694.45,48875.64,48623.38,48672.49,47.246988


In [4]:

Bitstamp_BTCUSD_1h.rename(columns={'close':'Close_Bitstamp', 'Volume BTC':'Volume_Bitstamp', 'open':'Open_Bitstamp',
                                 'high':'High_Bitstamp','low':'Low_Bitstamp'},inplace=True)

gemini_BTCUSD_1hr.rename(columns={'Close':'Close_gemini', 'Volume':'Volume_gemini','Open':'Open_gemini',
                                 'High':'High_gemini','Low':'Low_gemini'},inplace=True)

In [5]:
gemini_BTCUSD_1hr.head()

,Unix Timestamp,Date,Symbol,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini
0,1629504000000,2021-08-21 00:00:00,BTCUSD,49328.89,49383.31,49287.77,49312.94,10.695314
1,1629500400000,2021-08-20 23:00:00,BTCUSD,49242.38,49372.04,49064.75,49328.89,43.297139
2,1629496800000,2021-08-20 22:00:00,BTCUSD,49349.97,49397.11,48985.46,49242.38,89.115682
3,1629493200000,2021-08-20 21:00:00,BTCUSD,48672.49,49400.00,48663.12,49349.97,78.196379
4,1629489600000,2021-08-20 20:00:00,BTCUSD,48694.45,48875.64,48623.38,48672.49,47.246988


In [7]:
#BTC_prices  = gemini_BTCUSD_day[['Date','Close_gemini','Volume_gemini']].merge(Bitfinex_BTCUSD_1h[['date','Close_Bitfinex','Volume_Bitfinex']], left_on ='Date',right_on='date', how='left')
gemini_BTCUSD_1hr['Date'] = pd.to_datetime(gemini_BTCUSD_1hr['Date'])
Bitstamp_BTCUSD_1h['Date'] = pd.to_datetime(Bitstamp_BTCUSD_1h['date'])

BTC_prices  = gemini_BTCUSD_1hr[['Date','Open_gemini','High_gemini','Low_gemini','Close_gemini','Volume_gemini']].merge(Bitstamp_BTCUSD_1h[['Date','Open_Bitstamp','High_Bitstamp','Low_Bitstamp','Close_Bitstamp','Volume_Bitstamp']], on='Date', how='left')

BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp
0,2021-08-21 00:00:00,49328.89,49383.31,49287.77,49312.94,10.695314,49332.31,49367.72,49017.65,49017.65,25.678893
1,2021-08-20 23:00:00,49242.38,49372.04,49064.75,49328.89,43.297139,49257.02,49380.00,49043.88,49356.00,73.138954
2,2021-08-20 22:00:00,49349.97,49397.11,48985.46,49242.38,89.115682,49359.97,49400.00,48961.89,49240.81,104.919728
3,2021-08-20 21:00:00,48672.49,49400.00,48663.12,49349.97,78.196379,48681.50,49436.10,48681.50,49350.00,163.553410
4,2021-08-20 20:00:00,48694.45,48875.64,48623.38,48672.49,47.246988,48680.71,48863.57,48622.44,48649.68,102.674032


In [8]:
BTC_prices['Volume_BTC_gemini'] = BTC_prices['Volume_gemini']*BTC_prices['Close_gemini']

BTC_prices['Volume_BTC_Bitstamp'] = BTC_prices['Volume_Bitstamp']*BTC_prices['Close_Bitstamp']

BTC_prices['Close_prices'] = (BTC_prices['Close_gemini']+BTC_prices['Close_Bitstamp'])/2
BTC_prices['Open_prices'] = (BTC_prices['Open_gemini']+BTC_prices['Open_Bitstamp'])/2
BTC_prices['High_prices'] = (BTC_prices['High_gemini']+BTC_prices['High_Bitstamp'])/2
BTC_prices['Low_prices'] = (BTC_prices['Low_gemini']+BTC_prices['Low_Bitstamp'])/2
BTC_prices['Volume_BTC'] = BTC_prices['Volume_BTC_gemini']+BTC_prices['Volume_BTC_Bitstamp']
BTC_prices.head()

,Date,Open_gemini,High_gemini,Low_gemini,Close_gemini,Volume_gemini,Open_Bitstamp,High_Bitstamp,Low_Bitstamp,Close_Bitstamp,Volume_Bitstamp,Volume_BTC_gemini,Volume_BTC_Bitstamp,Close_prices,Open_prices,High_prices,Low_prices,Volume_BTC
0,2021-08-21 00:00:00,49328.89,49383.31,49287.77,49312.94,10.695314,49332.31,49367.72,49017.65,49017.65,25.678893,5.274174e+05,1.258719e+06,49165.295,49330.600,49375.515,49152.710,1.786136e+06
1,2021-08-20 23:00:00,49242.38,49372.04,49064.75,49328.89,43.297139,49257.02,49380.00,49043.88,49356.00,73.138954,2.135800e+06,3.609846e+06,49342.445,49249.700,49376.020,49054.315,5.745646e+06
2,2021-08-20 22:00:00,49349.97,49397.11,48985.46,49242.38,89.115682,49359.97,49400.00,48961.89,49240.81,104.919728,4.388268e+06,5.166332e+06,49241.595,49354.970,49398.555,48973.675,9.554601e+06
3,2021-08-20 21:00:00,48672.49,49400.00,48663.12,49349.97,78.196379,48681.50,49436.10,48681.50,49350.00,163.553410,3.858989e+06,8.071361e+06,49349.985,48676.995,49418.050,48672.310,1.193035e+07
4,2021-08-20 20:00:00,48694.45,48875.64,48623.38,48672.49,47.246988,48680.71,48863.57,48622.44,48649.68,102.674032,2.299629e+06,4.995059e+06,48661.085,48687.580,48869.605,48622.910,7.294687e+06


### Merge different datasets

Custom variable creating and summary data at HOURLY level is created

In [9]:
def data_manipulation(df):

    df['length_tweet'] = df['tweet_new2'].apply(len)
    df['Date'] = pd.to_datetime(df['created_date'])
    summarised = df.groupby(['Date'], as_index=False).agg({'neg':'mean','neu':'mean','pos':'mean','compound':'mean','retweet_count':'mean','like_count':'mean','author id':'count',
                                                                                  "unique_id": pd.Series.nunique, 'length_tweet':'mean','followers_count':'mean'})
    
    data_sum = df.groupby(['Date','sentiment'], as_index=False)['unique_id'].count()
    data_sum1 = data_sum.pivot(index='Date', columns='sentiment', values='unique_id')
    data_sum1['Total_hr_tweets'] = data_sum1['negative']+data_sum1['positive']+data_sum1['neutral']
    data_sum1 = data_sum1.reset_index()
    
    summarised = summarised.merge(data_sum1, on='Date', how='left')
 
    summarised['Bullish_Ratio'] = summarised['positive']/summarised['negative']
    summarised['Negative_share'] = summarised['negative']/summarised['Total_hr_tweets']
    summarised = summarised.rename(columns={'neg':'avg_neg_scr','neu':'avg_neu_scr','pos':'avg_pos_scr','compound':'avg_cmp_scr','retweet_count':'avg_retweet_cnt','like_count':'avg_like_cnt','author id':'tweet_vol',
                                                                                  "unique_id": 'uniq_tweet_vol', 'length_tweet':'avg_tweet_len','followers_count':'avg_followers_cnt'})
    summarised['Date']=pd.to_datetime(summarised['Date']).dt.tz_localize(None)
    return summarised

final_data_senti1=pd.read_csv('final_model_file1.csv')
summarised_1 = data_manipulation(final_data_senti1)

C:\Users\saggarwal\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
summarised_1.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-01-29 14:00:00,0.026134,0.874055,0.099817,0.186983,514.436786,3.322400,4817,2967,109.031347,4935.442364,544,2237,2036,4817,3.742647,0.112933
1,2021-01-29 15:00:00,0.026246,0.865902,0.107420,0.215853,369.950325,1.831236,4610,2762,114.789371,3516.211714,476,1992,2142,4610,4.500000,0.103254
2,2021-01-29 16:00:00,0.031139,0.873179,0.095692,0.166975,311.542531,2.335991,3762,2526,115.607390,4213.494152,524,1682,1556,3762,2.969466,0.139288
3,2021-01-29 17:00:00,0.030522,0.874927,0.094265,0.171298,272.722651,2.715510,3501,2431,119.879749,4451.978292,478,1531,1492,3501,3.121339,0.136532
4,2021-01-29 18:00:00,0.026299,0.853451,0.120250,0.244167,1105.064739,5.464176,3908,2354,119.820368,5923.160696,443,1554,1911,3908,4.313770,0.113357


In [12]:
final_data_senti1=pd.read_csv('final_data_senti_july31.csv')
summarised_2 = data_manipulation(final_data_senti1)
summarised_2.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-07-15 00:00:00,0.049885,0.796121,0.153611,0.157696,1479.907179,0.735409,15643,3525,85.171578,2241.458096,5105,4035,6503,15643,1.273849,0.326344
1,2021-07-15 01:00:00,0.049634,0.818682,0.131606,0.165034,1982.074331,1.287026,12525,3619,97.189701,3626.115689,4028,3116,5381,12525,1.335899,0.321597
2,2021-07-15 02:00:00,0.030379,0.723348,0.246211,0.273416,1858.430639,1.225073,16097,3512,85.522644,2408.319998,2968,3128,10001,16097,3.369609,0.184382
3,2021-07-15 03:00:00,0.030651,0.721518,0.247757,0.264724,1952.371707,0.651347,13360,3250,85.348278,2141.699476,2435,2823,8102,13360,3.327310,0.182260
4,2021-07-15 04:00:00,0.033003,0.788626,0.178372,0.229061,2057.914886,0.615020,11185,3300,99.155387,2105.588377,2288,3260,5637,11185,2.463724,0.204560


In [13]:
final_data_senti1=pd.read_csv('final_data_senti_aug15.csv')
summarised_3 = data_manipulation(final_data_senti1)
summarised_3.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-08-01 00:00:00,0.023358,0.881077,0.095568,0.191784,2325.665690,2.065020,7167,3639,115.041440,4123.976420,725,3511,2931,7167,4.042759,0.101158
1,2021-08-01 01:00:00,0.024499,0.863387,0.112114,0.230586,2748.378673,1.492870,8065,3695,117.721885,2315.055549,926,3434,3705,8065,4.001080,0.114817
2,2021-08-01 02:00:00,0.018218,0.872470,0.108934,0.228873,2852.383441,1.769658,7923,3986,117.483150,3857.863940,646,3845,3432,7923,5.312693,0.081535
3,2021-08-01 03:00:00,0.013876,0.871271,0.114852,0.256118,4266.560565,1.767608,11046,4859,114.758827,2295.338675,635,4527,5884,11046,9.266142,0.057487
4,2021-08-01 04:00:00,0.012710,0.904836,0.082455,0.166900,3900.253173,1.514911,15760,5275,103.828744,2028.063388,927,8174,6659,15760,7.183387,0.058820


In [14]:
summarised_all=summarised_1.append(summarised_2)
summarised_all=summarised_all.append(summarised_3)
summarised_all.shape

(1162, 17)

In [15]:
#summarised_all['created_date']=pd.to_datetime(summarised_all['created_date'])
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,avg_followers_cnt,negative,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share
0,2021-01-29 14:00:00,0.026134,0.874055,0.099817,0.186983,514.436786,3.322400,4817,2967,109.031347,4935.442364,544,2237,2036,4817,3.742647,0.112933
1,2021-01-29 15:00:00,0.026246,0.865902,0.107420,0.215853,369.950325,1.831236,4610,2762,114.789371,3516.211714,476,1992,2142,4610,4.500000,0.103254
2,2021-01-29 16:00:00,0.031139,0.873179,0.095692,0.166975,311.542531,2.335991,3762,2526,115.607390,4213.494152,524,1682,1556,3762,2.969466,0.139288
3,2021-01-29 17:00:00,0.030522,0.874927,0.094265,0.171298,272.722651,2.715510,3501,2431,119.879749,4451.978292,478,1531,1492,3501,3.121339,0.136532
4,2021-01-29 18:00:00,0.026299,0.853451,0.120250,0.244167,1105.064739,5.464176,3908,2354,119.820368,5923.160696,443,1554,1911,3908,4.313770,0.113357


In [16]:
summarised_all = summarised_all.merge(BTC_prices[['Date','Open_prices', 'High_prices','Low_prices','Close_prices','Volume_BTC']], left_on='Date', right_on='Date',how='left')
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-01-29 14:00:00,0.026134,0.874055,0.099817,0.186983,514.436786,3.322400,4817,2967,109.031347,...,2237,2036,4817,3.742647,0.112933,38374.715,38554.090,36555.395,37178.715,9.293677e+07
1,2021-01-29 15:00:00,0.026246,0.865902,0.107420,0.215853,369.950325,1.831236,4610,2762,114.789371,...,1992,2142,4610,4.500000,0.103254,37174.400,37634.140,36840.670,37154.310,6.039456e+07
2,2021-01-29 16:00:00,0.031139,0.873179,0.095692,0.166975,311.542531,2.335991,3762,2526,115.607390,...,1682,1556,3762,2.969466,0.139288,37163.115,37561.585,35819.085,36075.310,4.538283e+07
3,2021-01-29 17:00:00,0.030522,0.874927,0.094265,0.171298,272.722651,2.715510,3501,2431,119.879749,...,1531,1492,3501,3.121339,0.136532,36071.390,36454.320,35584.290,35746.085,4.412570e+07
4,2021-01-29 18:00:00,0.026299,0.853451,0.120250,0.244167,1105.064739,5.464176,3908,2354,119.820368,...,1554,1911,3908,4.313770,0.113357,35750.065,36032.380,34605.705,35208.460,6.315223e+07


In [18]:
summarised_all['Date'] = pd.to_datetime(summarised_all['Date'])
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,neutral,positive,Total_hr_tweets,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC
0,2021-01-29 14:00:00,0.026134,0.874055,0.099817,0.186983,514.436786,3.322400,4817,2967,109.031347,...,2237,2036,4817,3.742647,0.112933,38374.715,38554.090,36555.395,37178.715,9.293677e+07
1,2021-01-29 15:00:00,0.026246,0.865902,0.107420,0.215853,369.950325,1.831236,4610,2762,114.789371,...,1992,2142,4610,4.500000,0.103254,37174.400,37634.140,36840.670,37154.310,6.039456e+07
2,2021-01-29 16:00:00,0.031139,0.873179,0.095692,0.166975,311.542531,2.335991,3762,2526,115.607390,...,1682,1556,3762,2.969466,0.139288,37163.115,37561.585,35819.085,36075.310,4.538283e+07
3,2021-01-29 17:00:00,0.030522,0.874927,0.094265,0.171298,272.722651,2.715510,3501,2431,119.879749,...,1531,1492,3501,3.121339,0.136532,36071.390,36454.320,35584.290,35746.085,4.412570e+07
4,2021-01-29 18:00:00,0.026299,0.853451,0.120250,0.244167,1105.064739,5.464176,3908,2354,119.820368,...,1554,1911,3908,4.313770,0.113357,35750.065,36032.380,34605.705,35208.460,6.315223e+07


In [19]:
summarised_all['polarity'] = stats.gmean(summarised_all.iloc[:, [1,3]], axis=1)

In [20]:
summarised_all.head()
summarised_all.columns

Index(['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Total_hr_tweets', 'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity'],
      dtype='object')

In [21]:
summarised_all = summarised_all[summarised_all['Date'] >= '2021-07-01'][['Date', 'avg_neg_scr', 'avg_neu_scr', 'avg_pos_scr', 'avg_cmp_scr',
       'avg_retweet_cnt', 'avg_like_cnt', 'tweet_vol', 'uniq_tweet_vol',
       'avg_tweet_len', 'avg_followers_cnt', 'negative', 'neutral', 'positive',
       'Bullish_Ratio', 'Negative_share', 'Open_prices',
       'High_prices', 'Low_prices', 'Close_prices', 'Volume_BTC', 'polarity']]
summarised_all.to_csv('model_file_final_btc.csv')

In [22]:
summarised_all['pct_change'] = summarised_all.Close_prices.pct_change()

In [23]:
def direction(df):

    if (df['pct_change'] <0):
        return 0
    elif (df['pct_change'] >=0):
        return 1

summarised_all['direction']= summarised_all.apply(direction, axis = 1)

In [24]:
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,Bullish_Ratio,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction
58,2021-07-01 00:00:00,0.029129,0.836432,0.134196,0.303921,1419.020244,1.408293,4100,2226,131.943902,...,6.810496,0.083659,35040.355,35104.565,34716.810,34736.550,5.756833e+06,0.062521,NaN,NaN
59,2021-07-01 01:00:00,0.028587,0.845381,0.125752,0.280544,1900.329994,1.473011,3594,2041,131.055648,...,6.133987,0.085142,34746.335,35037.370,34733.715,34961.495,3.252340e+06,0.059957,0.006476,1.0
60,2021-07-01 02:00:00,0.025111,0.833500,0.141384,0.320735,1689.920310,1.349238,4003,2186,130.661254,...,7.796053,0.075943,34949.460,34960.680,34229.515,34320.675,5.981050e+06,0.059584,-0.018329,0.0
61,2021-07-01 03:00:00,0.028438,0.831342,0.139967,0.312858,1946.256461,0.811630,4024,2136,133.565358,...,6.780627,0.087227,34324.925,34419.610,34131.020,34131.020,3.109104e+06,0.063090,-0.005526,0.0
62,2021-07-01 04:00:00,0.034749,0.821632,0.142275,0.297292,1810.257934,1.342926,3718,2304,136.608392,...,5.000000,0.114578,34124.455,34346.205,34037.685,34338.375,5.322490e+06,0.070313,0.006075,1.0


### dow jones prices

Daily level market prices are merged with the summarised tweet dataset

In [27]:
dow=pd.read_csv('$DOWI_Barchart_Interactive_Chart_Intraday_60m_08_21_2021.csv', skiprows=[0,642])
dow.tail()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest
635,2021-08-20 13:00:00,35132.16,35155.60,35096.65,35098.28,-36.72,18653679,NaN
636,2021-08-20 14:00:00,35098.73,35130.90,35056.21,35085.53,-12.75,20315106,NaN
637,2021-08-20 15:00:00,35085.61,35154.06,35081.67,35117.69,32.16,49183911,NaN
638,2021-08-20 16:00:00,35117.06,35120.08,35116.27,35120.08,2.39,35539860,NaN
639,2021-08-20 17:00:00,35120.08,35120.08,35120.08,35120.08,0.00,227449,NaN


In [28]:
dow['Date']=pd.to_datetime(dow['Date Time'])
dow['index_change']=dow['Close'].pct_change()
dow.head()

,Date Time,Open,High,Low,Close,Change,Volume,Open Interest,Date,index_change
0,2021-05-11 12:00:00,34298.00,34305.31,34115.15,34145.67,0.00,34485086,NaN,2021-05-11 12:00:00,NaN
1,2021-05-11 13:00:00,34145.68,34303.58,34134.82,34269.57,123.90,32900368,NaN,2021-05-11 13:00:00,0.003629
2,2021-05-11 14:00:00,34270.29,34303.65,34182.43,34274.98,5.41,33717216,NaN,2021-05-11 14:00:00,0.000158
3,2021-05-11 15:00:00,34274.78,34337.85,34154.06,34270.17,-4.81,62506021,NaN,2021-05-11 15:00:00,-0.000140
4,2021-05-11 16:00:00,34269.88,34273.77,34269.16,34269.16,-1.01,37515412,NaN,2021-05-11 16:00:00,-0.000029


In [29]:
summarised_all=summarised_all.merge(dow[['Date','Close']], on='Date', how='left')
summarised_all['Close'] = summarised_all['Close'].fillna(method='ffill')
summarised_all['Close'] = summarised_all['Close'].fillna(34502)
summarised_all.head()

,Date,avg_neg_scr,avg_neu_scr,avg_pos_scr,avg_cmp_scr,avg_retweet_cnt,avg_like_cnt,tweet_vol,uniq_tweet_vol,avg_tweet_len,...,Negative_share,Open_prices,High_prices,Low_prices,Close_prices,Volume_BTC,polarity,pct_change,direction,Close
0,2021-07-01 00:00:00,0.029129,0.836432,0.134196,0.303921,1419.020244,1.408293,4100,2226,131.943902,...,0.083659,35040.355,35104.565,34716.810,34736.550,5.756833e+06,0.062521,NaN,NaN,34502.0
1,2021-07-01 01:00:00,0.028587,0.845381,0.125752,0.280544,1900.329994,1.473011,3594,2041,131.055648,...,0.085142,34746.335,35037.370,34733.715,34961.495,3.252340e+06,0.059957,0.006476,1.0,34502.0
2,2021-07-01 02:00:00,0.025111,0.833500,0.141384,0.320735,1689.920310,1.349238,4003,2186,130.661254,...,0.075943,34949.460,34960.680,34229.515,34320.675,5.981050e+06,0.059584,-0.018329,0.0,34502.0
3,2021-07-01 03:00:00,0.028438,0.831342,0.139967,0.312858,1946.256461,0.811630,4024,2136,133.565358,...,0.087227,34324.925,34419.610,34131.020,34131.020,3.109104e+06,0.063090,-0.005526,0.0,34502.0
4,2021-07-01 04:00:00,0.034749,0.821632,0.142275,0.297292,1810.257934,1.342926,3718,2304,136.608392,...,0.114578,34124.455,34346.205,34037.685,34338.375,5.322490e+06,0.070313,0.006075,1.0,34502.0


In [30]:
summarised_all.to_csv('input_model_file_new.csv')

In [31]:
summarised_all.shape

(1104, 25)